# Libraries

In [2]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser

# Webscraping

In [ ]:
# Initializing driver
driver = webdriver.Chrome()

In [ ]:
# Opening link to the Alltransit website
URL = r'https://alltransit.cnt.org/rankings/'
driver.get(URL)

In [ ]:
# Changing the settings of the table to show all cities with a population of over 100,000 
driver.find_element(By.XPATH, ".//select[@name='show-these']/option[@value='all']").click()
driver.find_element(By.XPATH, ".//select[@name='min-pop']/option[@value='100000']").click()

In [ ]:

# Initializing lists to use to create dataframe
ranks, names, scores, tcis, jobs, trips, routes, transitsheds, transitpercentage, populations = [],[],[],[],[],[],[],[],[],[]

# Finding the table & associated rows on the webpage
table = driver.find_element(By.XPATH, ".//table")
rows = [ row for row in table.find_elements(By.TAG_NAME, 'tr') ]

# For each row of data, gather each table cell into a list via comprehension
for row in rows[1:]:
    cells = [ cell.text for cell in row.find_elements(By.TAG_NAME, 'td') ]
    
    ranks += [int(cells[0])]
    names += [cells[1]]
    scores += [float(cells[2])]
    tcis += [float(cells[3])]
    jobs += [int(''.join([ char for char in cells[4] if char in '0123456789']))]
    trips += [int(''.join([ char for char in cells[5] if char in '0123456789']))]
    routes += [int(cells[6])]
    transitsheds += [int(cells[7][:-4])]
    transitpercentage += [round(float(cells[8][:-1])/100, 3)]
    populations += [int(''.join([ char for char in cells[9] if char in '0123456789']))]
    
    # Data cleaning while we are gathering data
    # ranks: extract cast as integer
    # names: extract as is
    # scores: extract cast as float
    # tcis: extract cast as float
    # jobs: extract only numeric characters via list comprehension, cast list into string via join method, cast as integer 
    # trips: extract only numeric characters via list comprehension, cast list into string via join method, cast as integer 
    # routes: extract cast as integer
    # transitsheds: extract all but the last 4 characters ' mi2', extract cast as integer
    # transitpercentage: extract all but last character '%', divide by 100 for decimal notation, round to remove floating point errors , extract cast as float
    # populations :extract only numeric characters via list comprehension, cast list into string via join method, cast as integer 

In [8]:
# Sanity Check
print(f'Row Counts: {[ len(element) for element in [ranks, names, scores, tcis, jobs, trips, routes, transitsheds, transitpercentage, populations] ]}')

Row Counts: [324, 324, 324, 324, 324, 324, 324, 324, 324, 324]


In [ ]:
# Creating the dataframe by aggregating data into a dictionary, then casting to a dataframe
alltransit_df=pd.DataFrame({
    'rank':ranks,
    'name':names,
    'score':scores,
    'tci':tcis,
    'jobs':jobs,
    'trips/week':trips,
    'routes':routes,
    'transit shed (mi2)':transitsheds,
    '%transit':transitpercentage,
    'population':populations,
})

In [10]:
alltransit_df.head(3)

,rank,name,score,tci,jobs,trips/week,routes,transit shed (mi2),%transit,population
0,1,"New York, NY",9.6,8.7,1296548,13960,19,44,0.544,8621501
1,2,"San Francisco, CA",9.3,7.4,596446,9616,20,35,0.312,851036
2,3,"Jersey City, NJ",9.2,5.6,828599,5272,3,26,0.478,287899


In [ ]:
# Saving data as scraping is no longer needed
alltransit_df.to_csv('alltransit100k.csv', index=False, encoding='utf-8')